In [2]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain.llms import OpenAI




In [3]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [4]:
GENERATIONS_PATH = "../datasets/manual_gpt3_50_50_generations_merged.csv"
MODEL1 = "gpt-3.5-turbo-0301"
MODEL2 = "gpt-3.5-turbo-instruct"

In [5]:
data = pd.read_csv(GENERATIONS_PATH)



if "decision" not in data.columns:
    data["decision"] = ""
if "similarity" not in data.columns:
    data["similarity"] = ""
if "confidence" not in data.columns:
    data["confidence"] = ""

data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,decision,similarity,confidence,structured_explanation
0,0,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:22.070Z,2023-10-17T10:25:25.422Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,True,FN,No Match,0%,100%,"[\n {\n ""attribute"": ""Title"",\n ""import..."
1,1,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:12.823Z,2023-10-17T10:25:17.121Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same one. They...",Match.,True,True,NONE,Match,95%,90%,"[\n {\n ""attribute"": ""Brand"",\n ""import..."
2,2,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:04.230Z,2023-10-17T10:25:07.905Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same. They are...",Match.,True,True,NONE,Match,100%,95%,"[{""attribute"":""Product Number"",""importance"":""0..."
3,3,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:56.337Z,2023-10-17T10:24:59.610Z,Q: Could you please tell me if the following t...,These two products are likely the same because...,Match.,True,True,NONE,Match,90%,80%,"[\n {\n ""attribute"": ""Brand"",\n ""import..."
4,4,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:48.374Z,2023-10-17T10:24:51.714Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same one. They...",Match.,True,True,NONE,Match,100%,95%,"[\n {\n ""attribute"": ""Brand"",\n ""import..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:58.234Z,2023-10-17T09:52:02.222Z,Q: Could you please tell me if the following t...,These two products are not the same one. The f...,No match.,False,False,NONE,No Match,0%,100%,Here are the attributes that I compared for re...
217,217,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:46.656Z,2023-10-17T09:51:54.143Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""Brand"",\n ""import..."
218,218,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:37.814Z,2023-10-17T09:51:42.110Z,Q: Could you please tell me if the following t...,These products are not the same one as they ar...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""category"",\n ""imp..."
219,219,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:27.877Z,2023-10-17T09:51:33.410Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""Camera Type"",\n ""..."


In [6]:


task = """Now output your decision as a simple 'Match' or 'No Match'. 
Also provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Decision: Match
Similarity: 75%
Confidence: 60%
"""


STEP_CONFIDENCE_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", task),
        ])





In [7]:
def get_confidence_score(start, end):
    llm = ChatOpenAI(model_name=MODEL1, temperature=0, max_tokens=150)
    chain = LLMChain(llm=llm, prompt=STEP_CONFIDENCE_PROMPT, callbacks=[handler])
    for i in range(start, end):
        print(i)
        whole_prompt = data["prompt"][i]
        user_prompt = "Q:" + whole_prompt.split("Q:")[-1]
        ai_answer = data["completion"][i]
        try:
            output = chain.run(user_prompt=user_prompt, ai_answer=ai_answer, callbacks=[handler])
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_CONFIDENCE_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        splitted_output = output.split("\n")
        decision = splitted_output[0].split(": ")[1]
        similarity = splitted_output[1].split(": ")[1]
        confidence = splitted_output[2].split(": ")[1]
        data["decision"][i] = decision
        data["similarity"][i] = similarity
        data["confidence"][i] = confidence



In [8]:
# get_confidence_score(40, len(data))
# data.to_csv(GENERATIONS_PATH, index=False)


In [9]:
task_structured_explanation = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95" "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00" "values":["missing","Graphite"]}]"""

task_structured_explanation_v2 = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95", "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00", "values":["missing","Graphite"]}]"""



STEP_EXPLANATION_STRUCTURED_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", "{task_structured_explanation}"),
        ])




In [10]:
if "structured_explanation" not in data.columns:
    data["structured_explanation"] = ""
if "structured_explanation_v2_instruct" not in data.columns:
    data["structured_explanation_v2_instruct"] = ""

In [11]:
def get_structured_explanation(start, end):
    if (MODEL2 == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL2, temperature=0)
    else:
        llm = ChatOpenAI(model_name=MODEL2, temperature=0)
    chain = LLMChain(llm=llm, prompt=STEP_EXPLANATION_STRUCTURED_PROMPT, callbacks=[handler])
    for i in range(start, end):
        print(i)
        whole_prompt = data["prompt"][i]
        user_prompt = "Q:" + whole_prompt.split("Q:")[-1]
        ai_answer = data["completion"][i]
        try:
            output = chain.run(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2, callbacks=[handler])
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_EXPLANATION_STRUCTURED_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        data["structured_explanation_v2_instruct"][i] = output



In [12]:
get_structured_explanation(0, len(data))

0
System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2: 
Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none
A:

AI: These two products are not the same because they have different titles and descriptions, and one refers to a specific refill for a notebook while the other refers to a general category of notebook accessories.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of 

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89819/3514648913.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2_instruct"][i] = output


System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: MINION DHR II 29 X 2.30 3C TERRA TR DD FOLD
Description: Refined and redesigned; ready to shred any lineRamped and siped tread to roll fast and track straight under brakingSingle-ply, DoubleDown, and DH casings available3C MaxxTerra, 3C MaxxGrip, and Super Tacky available on select modelsWide trail (WT) ca
Product 2: 
Title: Maxxis Minion DHR II 29\" DD/3C/TR
Description: Minion DHR II 29 x 2.35 tubeless ready tyre with Double down casing and 3C Maxx Terra.For our full range of Bike Accessories visit Evans Cycles
A:

AI: Yes, these two products are the same one. They both refer to the Maxxis Minion DHR II 29\ tire with Double Down casing and 3C Maxx Terra. The only difference is in the specific size and description details.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribu

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89819/3514648913.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2_instruct"][i] = output


System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Traveler's Company 014. CRAFT PAPER (64PG) MIDORI TRAVELER'S NOTEBOOK
Description: none
Product 2: 
Title: Refill #014: Kraft Paper Notebook
Description: Cardboard Paper notebook refill for the Traveler's Notebook.Various cuttings, stickers, photos, scribbles and more sophisticated drawings suit it well.Contains 64 pages and its dimensions (210 x 110 mm) make it suitable for the classic version, not f
A:

AI: Yes, these two products are the same. They are both refills for the Traveler's Notebook and have the same product number (#014) and description of being a kraft paper notebook with 64 pages.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the de

...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89819/3514648913.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2_instruct"][i] = output


System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Instax Mini Rainbow Film 10 Pack
Description: Brighten the place up with this pack of rainbow coloured film! Top tip: one of these would look fab in our Instax Mini Acrylic Photo Block!Includes pack of 10 shots
Product 2: 
Title: 1 Fujifilm Instax Square Film Black Frame
Description: none
A:

AI: These two products are not the same because they are different types of film for different types of cameras. Product 1 is for the Instax Mini camera and has rainbow colors, while Product 2 is for the Instax Square camera and has black frames.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the va

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89819/3514648913.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2_instruct"][i] = output


System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: DJI Mavic Mini Drone Fly More Combo
Description: Mavic Mini, the everyday flycam. Open your sky with the compact, yet powerful Mavic Mini. Weighing under 250 grams, together with the DJI Fly APP, you'll enjoy a simplified flying experience that is almost as light as the average smartphone.
Product 2: 
Title: Fujifilm Instax Mini Instant Film - Monochrome (10 Shots)
Description: Monochrome (Black & White) Instant Film for Fujifilm Mini Instant Cameras10 Shots per PackISO 800High Gloss FinishSuperb Grain QualityFast Image FormationGreater Resistance to Age-Induced ChangesSingle-plastic Composition for Easier Disposal
A:

AI: These two products are not the same one as they are from different categories and have different functionalities. The first product is a drone while the second product is instant film for a camera.
Human: Explain your decision in a struc

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_89819/3514648913.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2_instruct"][i] = output


In [13]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,decision,similarity,confidence,structured_explanation,structured_explanation_v2_instruct
0,0,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:22.070Z,2023-10-17T10:25:25.422Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,True,FN,No Match,0%,100%,"[\n {\n ""attribute"": ""Title"",\n ""import...","\n\n[{""attribute"":""title"",""importance"":""0.5"",""..."
1,1,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:12.823Z,2023-10-17T10:25:17.121Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same one. They...",Match.,True,True,NONE,Match,95%,90%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:04.230Z,2023-10-17T10:25:07.905Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same. They are...",Match.,True,True,NONE,Match,100%,95%,"[{""attribute"":""Product Number"",""importance"":""0...","\n\n[{""attribute"":""Brand"",""importance"":""0.05"",..."
3,3,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:56.337Z,2023-10-17T10:24:59.610Z,Q: Could you please tell me if the following t...,These two products are likely the same because...,Match.,True,True,NONE,Match,90%,80%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:48.374Z,2023-10-17T10:24:51.714Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same one. They...",Match.,True,True,NONE,Match,100%,95%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:58.234Z,2023-10-17T09:52:02.222Z,Q: Could you please tell me if the following t...,These two products are not the same one. The f...,No match.,False,False,NONE,No Match,0%,100%,Here are the attributes that I compared for re...,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
217,217,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:46.656Z,2023-10-17T09:51:54.143Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.1"",""..."
218,218,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:37.814Z,2023-10-17T09:51:42.110Z,Q: Could you please tell me if the following t...,These products are not the same one as they ar...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""category"",\n ""imp...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
219,219,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:27.877Z,2023-10-17T09:51:33.410Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""Camera Type"",\n ""...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."


In [14]:
data.to_csv(GENERATIONS_PATH, index=False)
